In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
%matplotlib inline

In [ ]:
sys1 = SymbolicSys.from_callback(lambda t, y, p: [-y[0]/t], 1)
sys1.exprs

In [ ]:
def vary(sys, kws, *, t_end=1e20):
    res = [sys.integrate((1, t_end), [1.0], atol=1e-50, nsteps=64000#, record_order=True
                         , **kw) for kw in kws]
    return res

In [ ]:
kws = [
    dict(integrator='cvode', method='adams', rtol=1e-10),
    dict(integrator='cvode', method='bdf', rtol=1e-10, first_step=1e-10),
    dict(integrator='gsl', method='msbdf', rtol=1e-10),
    dict(integrator='gsl', method='bsimp', rtol=1e-10),
    dict(integrator='odeint', method='bs', rtol=1e-10)
]

res1 = vary(sys1, kws)

In [ ]:
def plot_variation(res, kws, *, start_idx=0, fig_kw=None, plot_order=False):
    if fig_kw is None:
        fig_kw = dict(figsize=(16,6), dpi=150, sharey=True)
        
    if plot_order:
        from mpl_toolkits.axes_grid1 import host_subplot
        import mpl_toolkits.axisartist as aa
        
        fig = plt.figure(**fig_kw)
        axes = [host_subplot(1, len(res), i+1, figure=fig, axes_class=aa.Axes) for i in range(len(res))]
    else:
        fig, axes = plt.subplots(1, len(res), **fig_kw)
        
    for ax, r, kw in zip(axes, res, kws):
        #r.plot(ax=ax)
        rx = 1/r.xout
        relerr = np.abs(r.yout.squeeze() - rx)/rx
        ax.plot(r.xout[start_idx:], np.log10(relerr[start_idx:])
                , label=f"{kw['integrator']}, {kw['method']}"
                , linewidth=0.5#, marker='x'
               )
        ax.set_xscale('log')
        ax.set_yscale('linear')
        ax.set_ylabel('log10 |relative error|')

        random_walk = np.sqrt(1.0+np.array(range(r.xout.size)))  # non-systematic errors (no bias)
        #ax.plot(r.xout, random_walk*kw['rtol'], label='prognosis')

        ax.legend()
        ax.set_ylim([-16, -2])

        ax2 = ax.twinx()
        ax2.plot(r.xout[:-1], np.diff(r.xout), color='k', linewidth=0.5
                 #, ls='None', marker='.', markersize=0.5, alpha=0.3
                )
        ax2.set_yscale('log')
        ax2.set_ylim([1e-12, r.xout[-1]])
        ax2.set_ylabel('step size')

        if plot_order:
            ax3 = ax.twinx()        
            ax3.axis["right"] = ax3.new_fixed_axis(loc="right", offset=(60, 0))
            p3, = ax3.plot(r.xout, r.info['orders'], linewidth=0.5, color='tab:orange')
            ax3.axis["right"].label.set_color(p3.get_color())

            ax2.axis["right"].toggle(all=True)
            ax3.axis["right"].toggle(all=True)
            
        ax.set_title(f"{r.info['n_steps']=}\n{r.info['time_cpu']=:.3f} s\n ")

    fig.tight_layout(pad=1.08, rect=(-.05, -.05, 1.05, 1.05))

In [ ]:
plot_variation(res1, kws)

In [ ]:
sys2 = sys1.as_autonomous()
sys2.exprs

In [ ]:
plot_variation(vary(sys2, kws), kws)